In [1]:
import os
import openai
 
openai.api_key = os.environ["OPENAI_API_KEY"]
client = openai.Client(api_key=os.environ.get('OPENAI_API_KEY', 'INVALID'))

def llm(prompt, stop=["\n"]):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=stop
    )
    return response.choices[0].message.content

In [2]:
import yaml
import alfworld
import alfworld.agents.environment
with open('base_config.yaml') as reader:
    config = yaml.safe_load(reader)
    
split = "eval_out_of_distribution"
print(config["dataset"]["data_path"])

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]    
    return ob


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/miniconda3/envs/alfworld/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/miniconda3/envs/alfworld/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/miniconda3/envs/alfworld/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/alfworld/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
 

/Users/msolodko/Desktop/University_of_Waterloo/R2L-Lab/ReAct/data/json_2.1.1/train
Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:00<00:00, 862.19it/s] 

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [3]:
import json
folder = './prompts/'
prompt_file = 'alfworld_3prompts.json'
with open(folder + prompt_file, 'r') as f:
    d = json.load(f)

In [8]:
import sys

def alfworld_run(prompt, to_print=False, ob=''):
    init_prompt = prompt + ob + '\n>'
    prompt = ''
    # print("intial prompt", init_prompt)
    if to_print:
        print(ob)
        sys.stdout.flush()
    for i in range(1, 50):
        action = llm(init_prompt + prompt, stop=['\n']).strip() 
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        if action.startswith('think:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {action}\nObs {i}: {observation}')
            sys.stdout.flush()
        prompt += f' {action}\n{observation}\n>'
        if done:
            return reward
    return 0

In [9]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}
cnts = [0] * 6
rs = [0] * 6

for _ in range(134):
    ob, info = env.reset()
    ob = '\n'.join(ob[0].split('\n\n')[1:])
    name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])
    # print(name)
    # print(ob)
    for i, (k, v) in enumerate(prefixes.items()):
        if name.startswith(k):
            prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0'] + '\nHere is the task.\n'
            print(k, v)
            r = alfworld_run(prompt, ob=ob)
            rs[i] += r
            cnts[i] += 1
            break
    print(_+1, 'r', r, 'rs', rs, 'cnts', cnts, 'sum(rs)/sum(cnts)', sum(rs) / sum(cnts))
    print('------------\n')


pick_clean_then_place clean
1 r 0 rs [0, 0, 0, 0, 0, 0] cnts [0, 1, 0, 0, 0, 0] sum(rs)/sum(cnts) 0.0
------------

pick_and_place put
2 r 0 rs [0, 0, 0, 0, 0, 0] cnts [1, 1, 0, 0, 0, 0] sum(rs)/sum(cnts) 0.0
------------

pick_clean_then_place clean
3 r 0 rs [0, 0, 0, 0, 0, 0] cnts [1, 2, 0, 0, 0, 0] sum(rs)/sum(cnts) 0.0
------------

pick_heat_then_place heat
4 r True rs [0, 0, 1, 0, 0, 0] cnts [1, 2, 1, 0, 0, 0] sum(rs)/sum(cnts) 0.25
------------

look_at_obj examine
5 r True rs [0, 0, 1, 0, 1, 0] cnts [1, 2, 1, 0, 1, 0] sum(rs)/sum(cnts) 0.4
------------

pick_and_place put
6 r 0 rs [0, 0, 1, 0, 1, 0] cnts [2, 2, 1, 0, 1, 0] sum(rs)/sum(cnts) 0.3333333333333333
------------

look_at_obj examine
7 r 0 rs [0, 0, 1, 0, 1, 0] cnts [2, 2, 1, 0, 2, 0] sum(rs)/sum(cnts) 0.2857142857142857
------------

pick_cool_then_place cool
8 r 0 rs [0, 0, 1, 0, 1, 0] cnts [2, 2, 1, 1, 2, 0] sum(rs)/sum(cnts) 0.25
------------

pick_cool_then_place cool
9 r 0 rs [0, 0, 1, 0, 1, 0] cnts [2, 2, 1, 2, 

KeyboardInterrupt: 